## Import all the necessary packages

In [7]:
import string
import numpy as np
from PIL import Image
import os
from pickle import dump, load
import numpy as np
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers.merge import add
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout

In [8]:
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

C:\Users\daani\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


## Preprocessing the data

In [9]:
## Loading the document file and reading the contents inside the file into a string
def load_doc(fn):
    file = open(fn, 'r')
    text = file.read()
    file.close()
    return text

## Create a descriptions disctionary that maps images with a list of 5 captions
def all_img_captions(fn):
    file = load_doc(fn)
    capt = file.split('\n')
    d = {}
    for i in capt[:-1]:
        img, i = i.split('\t')
        if img[:-2] not in d:
            d[img[:-2]] = [i]
        else:
            d[img[:-2]].append(capt)
    return d

## Removing punctuations, converting all text to lowercase and removing words that contain numbers 
def cleaning_text(captions):
    table = str.maketrans('','',string.punctuation)
    for img,caps in captions.items():
        for i,img_caption in enumerate(caps):
            desc = ''
            for x in range(len(img_caption)):
                img_caption[x].replace("-"," ")
                desc = img_caption[x].split()
            #converts to lowercase
            desc = [word.lower() for word in desc]
            #remove punctuation from each token
            desc = [word.translate(table) for word in desc]
            #remove hanging 's and a 
            desc = [word for word in desc if(len(word)>1)]
            #remove tokens with numbers in them
            desc = [word for word in desc if(word.isalpha())]
            #convert back to string
            img_caption = ' '.join(desc)
            captions[img][i]= img_caption
    return captions

## Seperate all the unique words and create the vocabulary from all the descriptions
def text_vocabulary(text):
    v = set()
    for key in text.keys():
        [v.update(d.split()) for d in text[key]]
    return v

## Create a list of all the descriptions that have been preprocessed and store them into a file
def save_descriptions(d, fn):
    lines = list()
    for key, text_list in d.items():
        for i in text_list:
            lines.append(key + '\t' + i)
    n = "\n".join(lines)
    file = open(fn,"w")
    file.write(n)
    file.close()

In [10]:
# Set the path
dataset_text = "Flickr8k_text"
dataset_images = "Flickr8k_Dataset"

# Prepare the text data
fn = dataset_text + "/" + "Flickr8k.token.txt"

In [11]:
# Loading file containing data and mapping them into dictionary
desc = all_img_captions(fn)
print("Length of descriptions = " ,len(desc))

Length of descriptions =  8092


In [12]:
# cleaning the text
clean = cleaning_text(desc)

In [13]:
# building vocab
vocab = text_vocabulary(clean)
print("Length of vocab = " , len(vocab))

Length of vocab =  0


In [14]:
save_descriptions(clean, "descriptions.txt")